### Гайд на каверы (на английском):
https://docs.google.com/document/d/13_l1bd1Osgz7qlAZn-zhklCbHpVRk6bYOuAuB78qmsE/edit?usp=sharing

### Гайд на тренировку (на английском):
https://docs.google.com/document/d/13ebnzmeEBc6uzYCMt-QVFQk-whVrK4zw8k7_Lw3Bv_A/edit?usp=sharing

##Gradio версия нейросети. (может не работать на бесплатном тарифе)

In [ ]:
#@title Проверка видеокарты.
!nvidia-smi

In [ ]:
#@title Установка зависимостей
import os
import csv
import shutil
import tarfile
import subprocess
from pathlib import Path
from datetime import datetime
from IPython.display import clear_output
import ipywidgets as widgets
from IPython.utils import capture
from google.colab import drive

#@markdown Это приведет к принудительному обновлению зависимостей даже после того, как первоначальная установка, казалось бы, работала.
ForceUpdateDependencies = False #@param{type:"boolean"}
#@markdown Это приведет к использованию временного хранилища, поэтому зависимости будут загружаться каждый раз, а не на Диск. Не требуется, если только вам действительно не нужна память объемом 160 МБ. (Включено по умолчанию для нетренировочной колабы, чтобы повысить начальную скорость запуска)
ForceTemporaryStorage = True #@param{type:"boolean"}

with capture.capture_output() as cap:
    print(f'⚓ Установка Cloudflared')
    !wget -cq -O /content/cloudflared-linux-amd64.deb "https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb"
    !dpkg -i /content/cloudflared-linux-amd64.deb
    !rm /content/cloudflared-linux-amd64.deb
with capture.capture_output() as cap:
    %pip install -q gTTS
    %pip install -q elevenlabs
    %pip install -q stftpitchshift==1.5.1

# Монтирование диска
if not ForceTemporaryStorage:

    if not os.path.exists('/content/drive'):
        drive.mount('/content/drive')
    else:
        print('Диск уже смонтирован...')

def install_packages():
    packages = ['build-essential', 'python3-dev', 'ffmpeg', 'aria2']
    pip_packages = ['pip', 'setuptools', 'wheel', 'httpx==0.23.0', 'faiss-gpu', 'fairseq', 'gradio==3.34.0',
                    'ffmpeg', 'ffmpeg-python', 'praat-parselmouth', 'pyworld', 'numpy==1.23.5',
                    'numba==0.56.4', 'librosa==0.9.2', 'mega.py', 'gdown', 'onnxruntime', 'pyngrok==4.1.12']

    print("Обновление и установка системных пакетов...")
    for package in packages:
        print(f"Installing {package}...")
        subprocess.check_call(['apt-get', 'install', '-qq', '-y', package])

    print("Обновление и установка пакетов pip...")
    subprocess.check_call(['pip', 'install', '--upgrade'] + pip_packages)

    print('Пакеты актуальны.')

def scan_and_write(base_path, output_file):
    with open(output_file, 'w', newline='') as f:
        writer = csv.writer(f)
        for dirpath, dirs, files in os.walk(base_path):
            for filename in files:
                fname = os.path.join(dirpath, filename)
                try:
                    mtime = os.path.getmtime(fname)
                    writer.writerow([fname, mtime])
                except Exception as e:
                    print(f'Пропуск несуществующего файла {fname}: {str(e)}')
    print(f'Завершена запись временных меток файловой системы в {output_file}.')
clear_output()

def compare_files(old_file, new_file):
    old_files = {}
    new_files = {}

    with open(old_file, 'r') as f:
        reader = csv.reader(f)
        old_files = {rows[0]:rows[1] for rows in reader}

    with open(new_file, 'r') as f:
        reader = csv.reader(f)
        new_files = {rows[0]:rows[1] for rows in reader}

    removed_files = old_files.keys() - new_files.keys()
    added_files = new_files.keys() - old_files.keys()
    unchanged_files = old_files.keys() & new_files.keys()

    changed_files = {f for f in unchanged_files if old_files[f] != new_files[f]}

    for file in removed_files:
        print(f'Файл был удален: {file}')

    for file in changed_files:
        print(f'Файл обновлен: {file}')

    return list(added_files) + list(changed_files)

if ForceTemporaryStorage:
    file_path = '/content/CachedRVC.tar.gz'
else:
    file_path = '/content/drive/MyDrive/RVC_Cached/CachedRVC.tar.gz'

content_file_path = '/content/CachedRVC.tar.gz'
extract_path = '/'

def extract_wav2lip_tar_files():
    !wget https://github.com/777gt/EVC/raw/main/wav2lip-HD.tar.gz
    !wget https://github.com/777gt/EVC/raw/main/wav2lip-cache.tar.gz

    with tarfile.open('/content/wav2lip-cache.tar.gz', 'r:gz') as tar:
        for member in tar.getmembers():
            target_path = os.path.join('/', member.name)
            try:
                tar.extract(member, '/')
            except:
                pass

    with tarfile.open('/content/wav2lip-HD.tar.gz') as tar:
        tar.extractall('/content')

extract_wav2lip_tar_files()

if not os.path.exists(file_path):
    folder_path = os.path.dirname(file_path)
    os.makedirs(folder_path, exist_ok=True)
    print('No cached dependency install found. Attempting to download GitHub backup..')

    try:
        download_url = "https://github.com/kalomaze/QuickMangioFixes/releases/download/release3/CachedRVC.tar.gz"
        !wget -O $file_path $download_url
        print('Загрузка была успешна!')
    except Exception as e:
        print('Загрзука была прервана:', str(e))

        # Delete the failed download file
        if os.path.exists(file_path):
            os.remove(file_path)
        print('Не удалось загрузить файл удален. Продолжение резервного копирования вручную..')

if Path(file_path).exists():
    if ForceTemporaryStorage:
        print('Загрузка CachedRVC.tar.gz завершена.')
    else:
        print('CachedRVC.tar.gz найден на Google Диске. Приступаем к копированию и извлечению...')

    if ForceTemporaryStorage:
         pass
    else:
        shutil.copy(file_path, content_file_path)

    print('Начало операции резервного копирования...')

    with tarfile.open(content_file_path, 'r:gz') as tar:
        for member in tar.getmembers():
            target_path = os.path.join(extract_path, member.name)
            try:
                tar.extract(member, extract_path)
            except Exception as e:
                print('Не удалось извлечь файл (это ненормально)... мы вынуждены загрузить обновление, чтобы компенсировать это.')
                ForceUpdateDependencies = True
        print(f'Разаривирование {content_file_path} в {extract_path} готово.')

    if ForceUpdateDependencies:
        install_packages()
        ForceUpdateDependencies = False
else:
    print('CachedRVC.tar.gz не найден. Приступаем к созданию индекса всех текущих файлов...')
    scan_and_write('/usr/', '/content/usr_files.csv')

    install_packages()

    scan_and_write('/usr/', '/content/usr_files_new.csv')
    changed_files = compare_files('/content/usr_files.csv', '/content/usr_files_new.csv')

    with tarfile.open('/content/CachedRVC.tar.gz', 'w:gz') as new_tar:
        for file in changed_files:
            new_tar.add(file)
            print(f'Добавлено в tar: {file}')

    os.makedirs('/content/drive/MyDrive/RVC_Cached', exist_ok=True)
    shutil.copy('/content/CachedRVC.tar.gz', '/content/drive/MyDrive/RVC_Cached/CachedRVC.tar.gz')
    print('Обновленный CachedRVC.tar.gz скопирован на Google Диск..')
    print('Зависимости полностью обновлены; будущие пробеги должны быть быстрее.')
with capture.capture_output() as cap:
    %pip install gradio==3.34.0
    %pip install -q stftpitchshift==1.5.1
clear_output()

inf('\u2714 Готово','success', '300px')

In [ ]:
#@title Копирование Github репозитория
import os
import base64
from IPython.display import clear_output
import ipywidgets as widgets
from IPython.utils import capture

reeee=base64.b64decode(("V2Vi").encode('ascii')).decode('ascii')
weeee=base64.b64decode(("R1VJ").encode('ascii')).decode('ascii')
os.chdir('/content/')

def edit_file(file_path):
    temp_file_path = "/tmp/temp_file.py"
    changes_made = False
    with open(file_path, "r") as file, open(temp_file_path, "w") as temp_file:
        previous_line = ""
        for line in file:
            new_line = line.replace("value=160", "value=128")
            if new_line != line:
                print("Заменено 'value=160' with 'value=128'")
                changes_made = True
            line = new_line

            new_line = line.replace("crepe hop length: 160", "crepe hop length: 128")
            if new_line != line:
                print("Заменено 'crepe hop length: 160' на 'crepe hop length: 128'")
                changes_made = True
            line = new_line

            new_line = line.replace("value=0.88", "value=0.75")
            if new_line != line:
                print("Заменено 'value=0.88' на 'value=0.75'")
                changes_made = True
            line = new_line

            if "label=i18n(\"输入源音量包络替换输出音量包络融合比例，越靠近1越使用输出包络\")" in previous_line and "value=1," in line:
                new_line = line.replace("value=1,", "value=0.25,")
                if new_line != line:
                    print("Заменен 'value=1,' на 'value=0.25,' исходя из условий")
                    changes_made = True
                line = new_line

            if 'choices=["pm", "harvest", "dio", "crepe", "crepe-tiny", "mangio-crepe", "mangio-crepe-tiny"], # Fork Feature. Add Crepe-Tiny' in previous_line:
                if 'value="pm",' in line:
                    new_line = line.replace('value="pm",', 'value="mangio-crepe",')
                    if new_line != line:
                        print("Заменен 'value=\"pm\",' with 'value=\"mangio-crepe\",' исходя из условий")
                        changes_made = True
                    line = new_line

            temp_file.write(line)
            previous_line = line

    import shutil
    shutil.move(temp_file_path, file_path)

    if changes_made:
        print("Изменения внесены и файл успешно сохранен..")
    else:
        print("Никаких изменений не потребовалось.")
repo_path = f"/content/Retrieval-based-Voice-Conversion-{reeee}UI"
if not os.path.exists(repo_path):
    !git clone https://github.com/Mangio621/Mangio-RVC-Fork.git
    os.chdir('/content/Mangio-RVC-Fork')
    !git checkout bf0ffdbc35da09b57306e429c6deda84496948a1
    !wget https://github.com/kalomaze/Mangio-Kalo-Tweaks/raw/patch-1/Easier{weeee}.py
    os.chdir('/content/')
    !mv /content/Mangio-RVC-Fork /content/Retrieval-based-Voice-Conversion-{reeee}UI
    edit_file(f"/content/Retrieval-based-Voice-Conversion-{reeee}UI/infer-web.py")
    !mkdir -p /content/Retrieval-based-Voice-Conversion-{reeee}UI/audios
    !wget https://github.com/777gt/EVC/raw/main/someguy.mp3 -O /content/Retrieval-based-Voice-Conversion-{reeee}UI/audios/someguy.mp3
    !wget https://github.com/777gt/EVC/raw/main/somegirl.mp3 -O /content/Retrieval-based-Voice-Conversion-{reeee}UI/audios/somegirl.mp3
    !rm -rf /content/Retrieval-based-Voice-Conversion-{reeee}UI/il8n/en_US.json
    !wget https://github.com/NeuroDonu/RVC-Colab/blob/main/en_US.json -P /content/Retrieval-based-Voice-Conversion-{reeee}UI/il8n/
else:
    print(f"Репозиторий уже существует по адресу {repo_path}. Пропускаю клонирование.")
with capture.capture_output() as cap:
    !mkdir -p /content/Retrieval-based-Voice-Conversion-{reeee}UI/stats/
    !wget -q https://cdn.discordapp.com/attachments/945486970883285045/1114717554481569802/peppy-generator-388800-07722f17a188.json -O /content/Retrieval-based-Voice-Conversion-{reeee}UI/stats/peppy-generator-388800-07722f17a188.json
    !rm -rf /Retrieval-based-Voice-Conversion-{reeee}UI/torchcrepe
    !git clone https://github.com/maxrmorrison/torchcrepe.git
    !mv torchcrepe/torchcrepe Retrieval-based-Voice-Conversion-{reeee}UI/
    !rm -rf torchcrepe
os.chdir(f'/content/Retrieval-based-Voice-Conversion-{reeee}UI')
!mkdir -p pretrained uvr5_weights
clear_output()

def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)

inf('\u2714 Готово','success', '300px')

In [ ]:
#@title Загрузка базовой модели
!apt-get install aria2
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversion{reeee}UI/resolve/main/pretrained/D32k.pth -d /content/Retrieval-based-Voice-Conversion-{reeee}UI/pretrained -o D32k.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversion{reeee}UI/resolve/main/pretrained/D40k.pth -d /content/Retrieval-based-Voice-Conversion-{reeee}UI/pretrained -o D40k.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversion{reeee}UI/resolve/main/pretrained/D48k.pth -d /content/Retrieval-based-Voice-Conversion-{reeee}UI/pretrained -o D48k.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversion{reeee}UI/resolve/main/pretrained/G32k.pth -d /content/Retrieval-based-Voice-Conversion-{reeee}UI/pretrained -o G32k.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversion{reeee}UI/resolve/main/pretrained/G40k.pth -d /content/Retrieval-based-Voice-Conversion-{reeee}UI/pretrained -o G40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversion{reeee}UI/resolve/main/pretrained/G48k.pth -d /content/Retrieval-based-Voice-Conversion-{reeee}UI/pretrained -o G48k.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversion{reeee}UI/resolve/main/pretrained/f0D32k.pth -d /content/Retrieval-based-Voice-Conversion-{reeee}UI/pretrained -o f0D32k.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversion{reeee}UI/resolve/main/pretrained/f0D40k.pth -d /content/Retrieval-based-Voice-Conversion-{reeee}UI/pretrained -o f0D40k.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversion{reeee}UI/resolve/main/pretrained/f0D48k.pth -d /content/Retrieval-based-Voice-Conversion-{reeee}UI/pretrained -o f0D48k.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversion{reeee}UI/resolve/main/pretrained/f0G32k.pth -d /content/Retrieval-based-Voice-Conversion-{reeee}UI/pretrained -o f0G32k.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversion{reeee}UI/resolve/main/pretrained/f0G40k.pth -d /content/Retrieval-based-Voice-Conversion-{reeee}UI/pretrained -o f0G40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversion{reeee}UI/resolve/main/pretrained/f0G48k.pth -d /content/Retrieval-based-Voice-Conversion-{reeee}UI/pretrained -o f0G48k.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversion{reeee}UI/resolve/main/uvr5_weights/HP2-人声vocals+非人声instrumentals.pth -d /content/Retrieval-based-Voice-Conversion-{reeee}UI/uvr5_weights -o HP2-人声vocals+非人声instrumentals.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversion{reeee}UI/resolve/main/uvr5_weights/HP5-主旋律人声vocals+其他instrumentals.pth -d /content/Retrieval-based-Voice-Conversion-{reeee}UI/uvr5_weights -o HP5-主旋律人声vocals+其他instrumentals.pth

!wget https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/rmvpe.pt -P /content/Retrieval-based-Voice-Conversion-WebUI/
!wget https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/hubert_base.pt -P /content/Retrieval-based-Voice-Conversion-WebUI/

!git clone https://github.com/kalomaze/externalcolabcode.git /content/Retrieval-based-Voice-Conversion-WebUI/utils
clear_output()
def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)

inf('\u2714 Готово','success', '300px')

##Список моделей:
###Вы можете скачать по **любой** ссылке, если она принадлежит RVC. (Мега, Драйв и т. д.)

Самая большая организованная коллекция голосов на #voice-models в https://discord.gg/aihubbrasil

Таблица архива моделей, отсортированная по популярности: https://docs.google.com/spreadsheets/d/1tAUaQrEHYgRsm1Lvrnj14HFHDwJWl0Bd9x0QePewNco/

In [ ]:
#@title #Шаг 2. Загрузка модели
#@markdown Вставьте URL-путь к модели (Mega, Drive и т. д.) и запустите код
from mega import Mega
import os
import shutil
from urllib.parse import urlparse, parse_qs
import urllib.parse
from google.oauth2.service_account import Credentials
import gspread
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
import requests
import hashlib

def calculate_md5(file_path):
    hash_md5 = hashlib.md5()
    with open(file_path, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

scope = ['https://www.googleapis.com/auth/spreadsheets',
         'https://www.googleapis.com/auth/drive.file',
         'https://www.googleapis.com/auth/drive']

config_path = '/content/Retrieval-based-Voice-Conversion-WebUI/stats/peppy-generator-388800-07722f17a188.json'

if os.path.exists(config_path):
    creds = Credentials.from_service_account_file(config_path, scopes=scope)
    client = gspread.authorize(creds)
else:
    print("Файл учетных данных листа отсутствует.")

book = client.open("RealtimeRVCStats")
sheet = book.get_worksheet(3)
def update_sheet(url, filename, filesize, md5_hash, index_version):
    data = sheet.get_all_records()
    df = pd.DataFrame(data)

    if md5_hash in df['MD5 Hash'].values:
        idx = df[df['MD5 Hash'] == md5_hash].index[0]

        df.loc[idx, 'Download Counter'] = int(df.loc[idx, 'Download Counter']) + 1
        sheet.update_cell(idx+2, df.columns.get_loc('Download Counter') + 1, int(df.loc[idx, 'Download Counter']))

        alt_url_cols = [col for col in df.columns if 'Alt URL' in col]
        alt_url_values = [df.loc[idx, col_name] for col_name in alt_url_cols]

        if url not in alt_url_values and url != df.loc[idx, 'URL']:
            for col_name in alt_url_cols:
                if df.loc[idx, col_name] == '':
                    df.loc[idx, col_name] = url
                    sheet.update_cell(idx+2, df.columns.get_loc(col_name) + 1, url)
                    break
    else:
        new_row_dict = {'URL': url, 'Download Counter': 1, 'Filename': filename,
                        'Filesize (.pth)': filesize, 'MD5 Hash': md5_hash, 'RVC Version': index_version}

        alt_url_cols = [col for col in df.columns if 'Alt URL' in col]
        for col in alt_url_cols:
            new_row_dict[col] = ''  

        new_row_dict = {key: str(value) if key not in ['Download Counter', 'Filesize (.pth)'] else value for key, value in new_row_dict.items()}

        ordered_row = [new_row_dict.get(col, '') for col in df.columns]
        sheet.append_row(ordered_row, value_input_option='RAW')

condition1 = False
condition2 = False
already_downloaded = False

from IPython.utils import capture

with capture.capture_output() as cap:
    !rm -rf /content/unzips/
    !rm -rf /content/zips/
    !mkdir /content/unzips
    !mkdir /content/zips

def sanitize_directory(directory):
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if os.path.isfile(file_path):
            if filename == ".DS_Store" or filename.startswith("._"):
                os.remove(file_path)
        elif os.path.isdir(file_path):
            sanitize_directory(file_path)

url = 'https://huggingface.co/kalomaze/KanyeV2_Redux/resolve/main/KanyeV2_REDUX_40khz.zip'  #@param {type:"string"}
model_zip = urlparse(url).path.split('/')[-2] + '.zip'
model_zip_path = '/content/zips/' + model_zip

private_model = False #@param{type:"boolean"}

if url != '':
    MODEL = "" 
    !mkdir -p /content/Retrieval-based-Voice-Conversion-WebUI/logs/$MODEL
    !mkdir -p /content/zips/
    !mkdir -p /content/Retrieval-based-Voice-Conversion-WebUI/weights/

    if "drive.google.com" in url:
        !gdown $url --fuzzy -O "$model_zip_path"
    elif "/blob/" in url:
        url = url.replace("blob", "resolve")
        print("Разрешенный URL-адрес:", url)  
        !wget "$url" -O "$model_zip_path"
    elif "mega.nz" in url:
        m = Mega()
        print("Начало скачивание с MEGA....")
        m.download_url(url, '/content/zips')
    elif "/tree/main" in url:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        temp_url = ''
        for link in soup.find_all('a', href=True):
            if link['href'].endswith('.zip'):
                temp_url = link['href']
                break
        if temp_url:
            url = temp_url
            print("Обновленный URL:", url) 
            url = url.replace("blob", "resolve")
            print("Решенный URL:", url) 

            if "huggingface.co" not in url:
                url = "https://huggingface.co" + url

            !wget "$url" -O "$model_zip_path"
        else:
            print("Не найден .zip файл на этой странице.")
    else:
        !wget "$url" -O "$model_zip_path"

    for filename in os.listdir("/content/zips"):
        if filename.endswith(".zip"):
            zip_file = os.path.join("/content/zips", filename)
            shutil.unpack_archive(zip_file, "/content/unzips", 'zip')

sanitize_directory("/content/unzips")

def find_pth_file(folder):
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.endswith(".pth"):
                file_name = os.path.splitext(file)[0]
                if file_name.startswith("G_") or file_name.startswith("P_"):
                    config_file = os.path.join(root, "config.json")
                    if os.path.isfile(config_file):
                        print("Обнаружен устаревший .pth! Это несовместимо с методом RVC. Найдите эквивалентную модель RVC!")
                    continue  
                file_path = os.path.join(root, file)
                if os.path.getsize(file_path) > 100 * 1024 * 1024:  
                    print("Пропуск непригодного для использования файла обучения:", file)
                    continue  
                return file_name
    return None

MODEL = find_pth_file("/content/unzips")
if MODEL is not None:
    print("найден .pth файл:", MODEL + ".pth")
else:
    print("Ошибка: не удалось найти действительный файл .pth в извлеченном zip-архиве.")
    print("Если выше появляется ошибка «Доступ запрещен», попробуйте один из альтернативных URL-адресов в Google Таблицах для этой модели.")
    MODEL = ""
    global condition3
    condition3 = True

index_path = ""

def find_version_number(index_path):
    if condition2 and not condition1:
        if file_size >= 55180000:
            return 'RVC v2'
        else:
            return 'RVC v1'

    filename = os.path.basename(index_path)

    if filename.endswith("_v2.index"):
        return 'RVC v2'
    elif filename.endswith("_v1.index"):
        return 'RVC v1'
    else:
        if file_size >= 55180000:
            return 'RVC v2'
        else:
            return 'RVC v1'

if MODEL != "":
    for root, dirs, files in os.walk('/content/unzips'):
        for file in files:
            file_path = os.path.join(root, file)
            if file.endswith(".index"):
                print("Найден файл индекса:", file)
                condition1 = True
                logs_folder = os.path.join('/content/Retrieval-based-Voice-Conversion-WebUI/logs', MODEL)
                os.makedirs(logs_folder, exist_ok=True) 

                if file.endswith(".index"):
                    identical_index_path = os.path.join(logs_folder, file)
                    if os.path.exists(identical_index_path):
                        os.remove(identical_index_path)

                shutil.move(file_path, logs_folder)
                index_path = os.path.join(logs_folder, file) 

            elif "G_" not in file and "D_" not in file and file.endswith(".pth"):
                destination_path = f'/content/Retrieval-based-Voice-Conversion-WebUI/weights/{MODEL}.pth'
                if os.path.exists(destination_path):
                    print("You already downloaded this model. Re-importing anyways..")
                    already_downloaded = True
                shutil.move(file_path, destination_path)
                condition2 = True
                if already_downloaded is False and os.path.exists(config_path):
                    file_size = os.path.getsize(destination_path)
                    md5_hash = calculate_md5(destination_path)
                    index_version = find_version_number(index_path)

if condition1 is False:
    logs_folder = os.path.join('/content/Retrieval-based-Voice-Conversion-WebUI/logs', MODEL)
    os.makedirs(logs_folder, exist_ok=True)

if condition2 and not condition1:
    print("Модель частично импортирована! При загрузке модели не найден индексный файл. Возможно, автор забыл добавить индексный файл.")
    if already_downloaded is False and os.path.exists(config_path) and not private_model:
        update_sheet(url, MODEL, file_size, md5_hash, index_version)

elif condition1 and condition2:
    print("Модель успешна импортирована!")
    if already_downloaded is False and os.path.exists(config_path) and not private_model:
        update_sheet(url, MODEL, file_size, md5_hash, index_version)

elif condition3:
    pass 
else:
    print("URL-адрес не может быть пустым. Если вы не хотите загружать модель сейчас, просто пропустите этот шаг.")

with capture.capture_output() as cap:
    def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)
    !rm -r /content/unzips/
    !rm -r /content/zips/
clear_output()

inf('\u2714 Готово','success', '300px')

#Step 3. Запустите графический интерфейс, затем откройте общедоступный URL-адрес. Это будет выглядеть так:
![alt text](https://i.imgur.com/ZjuyG29.png)

In [ ]:
import os
import time
import fileinput
from subprocess import getoutput
import sys
from IPython.utils import capture
from IPython.display import display, HTML, clear_output
%cd /content/Retrieval-based-Voice-Conversion-{reeee}UI/

#@markdown Оставьте эту опцию включенной, чтобы использовать упрощенный и простой интерфейс.
#@markdown <br>В противном случае будет использоваться расширенный вариант, который вы видите в руководстве на YouTube.
ezmode = True #@param{type:"boolean"}
#@markdown Вы можете попробовать использовать Cloudflare в качестве туннеля вместо gradio.live, если получаете ошибки подключения.
tunnel = "gradio" #@param ["cloudflared", "gradio"]

if ezmode:
  if tunnel == "cloudflared":
    for line in fileinput.FileInput(f'Easier{weeee}.py', inplace=True):
      if line.strip() == 'app.queue(concurrency_count=511, max_size=1022).launch(share=True, quiet=True)':
        line = f'        app.queue(concurrency_count=511, max_size=1022).launch(quiet=True)\n'
      sys.stdout.write(line)
    !pkill cloudflared
    time.sleep(4)
    !nohup cloudflared tunnel --url http://localhost:7860 > /content/srv.txt 2>&1 &
    time.sleep(4)
    !grep -o 'https[^[:space:]]*\.trycloudflare.com' /content/srv.txt >/content/srvr.txt
    time.sleep(2)
    srv=getoutput('cat /content/srvr.txt')
    display(HTML('<h1>Your <span style="color:orange;">Cloudflare URL</span> is printed below! Click the link once you see "Running on local URL".</span></h1><br><h2><a href="' + srv + '" target="_blank">' + srv + '</a></h2>'))
    !rm /content/srv.txt /content/srvr.txt
  elif tunnel == "gradio":
    for line in fileinput.FileInput(f'Easier{weeee}.py', inplace=True):
      if line.strip() == 'app.queue(concurrency_count=511, max_size=1022).launch(quiet=True)':
        line = f'        app.queue(concurrency_count=511, max_size=1022).launch(share=True, quiet=True)\n'
      sys.stdout.write(line)
  !python3 Easier{weeee}.py --colab --pycmd python3
else:
    !python3 infer-web.py --colab --pycmd python3

* Для исходного графического интерфейса RVC посетите: https://github.com/RVC-Project/Retrieval-based-Voice-Conversion-WebUI
* Если вам нужно обучить модель, посетите: https://colab.research.google.com/github/NeuroDonu/RVC-Colab/blob/main/RVC.ipynb

#Остальное

In [ ]:
#@markdown #Загрузите файлы (или сделайте это через панель Colab)
#@markdown Запустите эту ячейку, чтобы загрузить вокальные файлы, которые вы хотите использовать (или заархивировать файлы, содержащие аудио), в свой Colab.. <br>
#@markdown Альтернативно вы можете загрузить файлы с панели файлов Colab, как показано на видео, но это должно быть более удобно. Этот метод может не работать на iOS.

from google.colab import files
from IPython.display import display, Javascript
import os
import shutil
import zipfile
import ipywidgets as widgets

target_dir = '/content/Retrieval-based-Voice-Conversion-WebUI/audios/'
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

uploaded = files.upload()

for fn in uploaded.keys():
    if fn.endswith('.zip'):
        zip_path = os.path.join(target_dir, fn)
        with open(zip_path, 'wb') as f:
            f.write(uploaded[fn])

        unzip_dir = os.path.join(target_dir, fn[:-4]) 

        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(unzip_dir)

        if os.path.exists(zip_path):
            os.remove(zip_path)

        print('Архив "{name}" разархивирован и удален. Файлы находятся тут: {folder}'.format(name=fn, folder=unzip_dir))

        for extracted_file in os.listdir(unzip_dir):
            extracted_file_path = os.path.join(unzip_dir, extracted_file)
            extracted_file_length = os.path.getsize(extracted_file_path)

            extracted_file_label = widgets.HTML(
                value='Извлеченный файл "{name}" с весом {length} байт'.format(name=extracted_file, length=extracted_file_length)
            )
            display(extracted_file_label)

            extracted_file_path_text = widgets.HTML(
                value='Файл сохранен тут: <a href="{}" target="_blank">{}</a>'.format(extracted_file_path, extracted_file_path)
            )

            extracted_copy_button = widgets.Button(description='Copy')
            extracted_copy_button_file_path = extracted_file_path  # Make a local copy of the file path

            def copy_to_clipboard(b):
                js_code = '''
                    const el = document.createElement('textarea');
                    el.value = "{path}";
                    el.setAttribute('readonly', '');
                    el.style.position = 'absolute';
                    el.style.left = '-9999px';
                    document.body.appendChild(el);
                    el.select();
                    document.execCommand('copy');
                    document.body.removeChild(el);
                '''
                display(Javascript(js_code.format(path=extracted_copy_button_file_path)))

            extracted_copy_button.on_click(copy_to_clipboard)
            display(widgets.HBox([extracted_file_path_text, extracted_copy_button]))

        continue

    file_path = os.path.join(target_dir, fn)
    with open(file_path, 'wb') as f:
        f.write(uploaded[fn])

    file_length = len(uploaded[fn])
    file_label = widgets.HTML(
        value='Загруженный пользователем файл "{name}" с весом {length} байтов'.format(name=fn, length=file_length)
    )
    display(file_label)

    if fn.endswith('.pth') or fn.endswith('.index'):
        warning_text = widgets.HTML(
            value='<b style="color: red;">Предупреждение:</b> Вы загружаете файл модели не в то место. Убедитесь, что он загружен в правильное место.'
        )
        display(warning_text)

    file_path_text = widgets.HTML(
        value='Файл сохранен тут: <a href="{}" target="_blank">{}</a>'.format(file_path, file_path)
    )

    copy_button = widgets.Button(description='Copy')
    copy_button_file_path = file_path

    def copy_to_clipboard(b):
        js_code = '''
            const el = document.createElement('textarea');
            el.value = "{path}";
            el.setAttribute('readonly', '');
            el.style.position = 'absolute';
            el.style.left = '-9999px';
            document.body.appendChild(el);
            el.select();
            document.execCommand('copy');
            document.body.removeChild(el);
        '''
        display(Javascript(js_code.format(path=copy_button_file_path)))

    copy_button.on_click(copy_to_clipboard)
    display(widgets.HBox([file_path_text, copy_button]))

for fn in uploaded.keys():
    if os.path.exists(os.path.join("/content/", fn)):
        os.remove(os.path.join("/content/", fn))

In [ ]:
#@markdown ##Нажмите здесь, чтобы импортировать ZIP-архив АУДИО ФАЙЛОВ.
#@markdown Свяжите URL-путь с аудиофайлами (Mega, Drive и т. д.) и запустите код
url = 'INSERTURLHERE'  #@param {type:"string"}

import subprocess
import os
import shutil
from urllib.parse import urlparse, parse_qs
from google.colab import output
from google.colab import drive

mount_to_drive = True
mount_path = '/content/drive/MyDrive'

def mount(gdrive=False):
    if gdrive:
        if not os.path.exists("/content/drive/MyDrive"):
            try:
                drive.mount("/content/drive", force_remount=True)
            except:
                drive._mount("/content/drive", force_remount=True)
    else:
        pass

mount(mount_to_drive)

def check_package_installed(package_name):
    command = f"pip show {package_name}"
    result = subprocess.run(command.split(), stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    return result.returncode == 0

def install_package(package_name):
    command = f"pip install {package_name} --quiet"
    subprocess.run(command.split(), stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

if not check_package_installed("mega.py"):
    install_package("mega.py")

from mega import Mega
import os
import shutil
from urllib.parse import urlparse, parse_qs
import urllib.parse
from IPython.utils import capture

with capture.capture_output() as cap:
    !rm -rf /content/unzips/
    !rm -rf /content/zips/
    !mkdir /content/unzips
    !mkdir /content/zips

def sanitize_directory(directory):
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if os.path.isfile(file_path):
            if filename == ".DS_Store" or filename.startswith("._"):
                os.remove(file_path)
        elif os.path.isdir(file_path):
            sanitize_directory(file_path)

audio_zip = urlparse(url).path.split('/')[-2] + '.zip'
audio_zip_path = '/content/zips/' + audio_zip

if url != '':
    if "drive.google.com" in url:
        !gdown $url --fuzzy -O "$audio_zip_path"
    elif "mega.nz" in url:
        m = Mega()
        m.download_url(url, '/content/zips')
    else:
        !wget "$url" -O "$audio_zip_path"

    for filename in os.listdir("/content/zips"):
        if filename.endswith(".zip"):
            zip_file = os.path.join("/content/zips", filename)
            shutil.unpack_archive(zip_file, "/content/unzips", 'zip')

sanitize_directory("/content/unzips")

!mkdir -p /content/Retrieval-based-Voice-Conversion-WebUI/audios
for filename in os.listdir("/content/unzips"):
    if filename.endswith((".wav", ".mp3", ".m4a", ".flac")):
        audio_file = os.path.join("/content/unzips", filename)
        destination_file = os.path.join("/content/Retrieval-based-Voice-Conversion-WebUI/audios", filename)
        shutil.copy2(audio_file, destination_file)
        if os.path.exists(destination_file):
            print(f"Копирование успешно: {destination_file}")
        else:
            print(f"Копирование не успешно: {audio_file}")

with capture.capture_output() as cap:
    !rm -r /content/unzips/
    !rm -r /content/zips/
    def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)
clear_output()

inf('\u2714 Готово','success', '300px')

#UVR для вас

##Метод Colab UVR (MDX-Net)
Нижеследующее позволяет использовать следующие модели, рекомендованные для разделения акапелл из вашей музыки:
- Kim vocal 1
- Kim vocal 2 (более высокое качество, но может содержать больше бэк-вокала, который необходимо изолировать с помощью модели караоке)

Или для достижения наилучших инструментальных результатов вы можете позже сделать:
- Inst HQ 1

Реверберацию следует удалить с помощью Reverb HQ. С другими оставшимися эффектами эха можно справиться с помощью совместной лаборатории VR Architecture UVR, ссылка на которую приведена ниже, с использованием моделей De-Echo. (или сделано с помощью местного UVR)

In [ ]:
initialised = True
from time import sleep
from google.colab import output
from google.colab import drive

import sys
import os
import shutil
import psutil
import glob

mount_to_drive = True
mount_path = '/content/drive/MyDrive'

ai = 'https://github.com/kae0-0/Colab-for-MDX_B'
ai_version = 'https://github.com/kae0-0/Colab-for-MDX_B/raw/main/v'
onnx_list = 'https://raw.githubusercontent.com/kae0-0/Colab-for-MDX_B/main/onnx_list'
#@markdown Опция «ForceUpdate» обновит модели путем полной переустановки.
ForceUpdate = False #@param {type:"boolean"}
class h:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')
    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout
def get_size(bytes, suffix='B'): # read ram
    global svmem
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f'{bytes:.2f}{unit}{suffix}'
        bytes /= factor
    svmem = psutil.virtual_memory()
def console(t):
    get_ipython().system(t)
def LinUzip(file): # unzip call linux, force replace
    console(f'unzip -o {file}')
#-------------------------------------------------------
def getONNX():
    console(f'wget {onnx_list} -O onnx_list')
    _onnx = open("onnx_list", "r")
    _onnx = _onnx.readlines()
    os.remove('onnx_list')
    for model in _onnx:
        _model = sanitize_filename(os.path.basename(model))
        console(f'wget {model}')
        LinUzip(_model)
        os.remove(_model)

def getDemucs(_path):
    #https://dl.fbaipublicfiles.com/demucs/v3.0/demucs_extra-3646af93.th
    root = "https://dl.fbaipublicfiles.com/demucs/v3.0/"
    model = {
        'demucs_extra': '3646af93'
    }
    for models in zip(model.keys(),model.values()):
        console(f'wget {root+models[0]+"-"+models[1]}.th -O {models[0]}.th')
    for _ in glob.glob('*.th'):
        if os.path.isfile(os.path.join(os.getcwd(),_path,_)):
            os.remove(os.path.join(os.getcwd(),_path,_))
        shutil.move(_,_path)

def mount(gdrive=False):
    if gdrive:
        if not os.path.exists("/content/drive/MyDrive"):
            try:
                drive.mount("/content/drive", force_remount=True)
            except:
                drive._mount("/content/drive", force_remount=True)
    else:
        pass

mount(mount_to_drive)

def toPath(path='local'):
    if path == 'local':
        os.chdir('/content')
    elif path == 'gdrive':
        os.chdir(mount_path)

def update():
    with h():
        console(f'wget {ai_version} -O nver')
    f = open('nver', 'r')
    nver = f.read()
    f = open('v', 'r')
    cver = f.read()
    if nver != cver or ForceUpdate:
        print('Найдено новое обновление! {}'.format(nver))
        os.chdir('../')
        print('Обновление ИИ...',end=' ')
        with h():
            console(f'git clone {ai} temp_MDX_Colab')
            console('cp -a temp_MDX_Colab/* MDX_Colab/')
            console('rm -rf temp_MDX_Colab')
        print('done')
        os.chdir('MDX_Colab')
        print('Обновление моделей...', end=' ')
        with h():
            #getDemucs('model/')
            getONNX()
        print('done')
        output.clear()
        os.remove('v')
        os.rename("nver",'v')
        #os.chdir(f'{os.path.join(mount_path,"MDX_Colab")}')
    else:
        os.remove('nver')
        print('Использование последней версии.')

def past_installation():
    return os.path.exists('MDX_Colab')

def LoadMDX():
    console(f'git clone {ai} MDX_Colab')

#-------------------------------------------------------
# install requirements
print('Установка зависимостей, займет 45 секунд....',end=' ')

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    svmem = psutil.virtual_memory()
    gpu_runtime = False
    with h():
        console('pip3 install onnxruntime==1.14.1')
else:
    gpu_runtime = True
    with h():
        console('pip3 install onnxruntime-gpu==1.14.1')
with h():
    deps = [
            'pathvalidate',
            'youtube-dl',
            'django'
    ]
    for dep in deps:
        console('pip3 install {}'.format(dep))
# import modules
#console('pip3 install torch==1.13.1')
console('pip3 install soundfile==0.12.1')
console('pip3 install librosa==0.9.1')
from pathvalidate import sanitize_filename
print('done')
if not gpu_runtime:
    print(f'ГП отключен. У вас есть {get_size(svmem.total)} ОЗУ.\nОбработка будет невероятно медленной. 😈')
elif gpu_info.find('Tesla T4') >= 0:
    print('У вас ГП Tesla T4.. (скорости около  10-25 итераций/с))')
elif gpu_info.find('Tesla P4') >= 0:
    print('У вас ГП a Tesla P4 . (скорости около  8-22 итераций/с))')
elif gpu_info.find('Tesla K80') >= 0:
    print('У вас ГП Tesla K80. (Это обычный графический процессор, скорости около 2-10 итераций/с)')
elif gpu_info.find('Tesla P100') >= 0:
    print('У вас ГП Tesla P100. (Это второй по быстродействию графический процессор, скорость около  итераций/с))')
else:
    if gpu_runtime:
        print('У вас неизвестный ГП. Пожалуйста, сообщите, какой ГП вы получили!')
        !nvidia-smi

#console('pip3 install demucs')
#-------------------------------------------------------
# Scripting
mount(mount_to_drive)
toPath('gdrive' if mount_to_drive else 'local')
#check for MDX existence
if not past_installation():
    print('Первая установка займет около 3-6 минут..\nДля этого требуется около 2–3 ГБ свободного места на Gdrive..\nПожалуйста, постарайтесь не прерывать процесс установки!')
    print('Загрузка ИИ...',end=' ')
    with h():
        LoadMDX()
    os.chdir('MDX_Colab')
    print('Готово')

    print('Загрузка моделей...',end=' ')
    with h():
        #getDemucs('model/')
        getONNX()
    if os.path.isfile('onnx_list'):
        os.remove('onnx_list')
    print('Готово')

else:
    os.chdir('MDX_Colab')
    update()

################
#outro
print('Успешно!')

In [ ]:
#@markdown ##Нажмите здесь, чтобы импортировать ZIP-архив АУДИО ФАЙЛОВ (для изоляции).
#@markdown Или вы можете использовать ячейку ниже для прямой загрузки файлов (что более удобно) <br> <br>
#@markdown Свяжите URL-путь с аудиофайлами (Mega, Drive и т. д.) и запустите код.
url = 'INSERTURLHERE'  #@param {type:"string"}

import subprocess
import os
import shutil
from urllib.parse import urlparse, parse_qs
from google.colab import output
from google.colab import drive


mount_to_drive = True
mount_path = '/content/drive/MyDrive'

def mount(gdrive=False):
    if gdrive:
        if not os.path.exists("/content/drive/MyDrive"):
            try:
                drive.mount("/content/drive", force_remount=True)
            except:
                drive._mount("/content/drive", force_remount=True)
    else:
        pass

mount(mount_to_drive)

def check_package_installed(package_name):
    command = f"pip show {package_name}"
    result = subprocess.run(command.split(), stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    return result.returncode == 0

def install_package(package_name):
    command = f"pip install {package_name} --quiet"
    subprocess.run(command.split(), stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

if not check_package_installed("mega.py"):
    install_package("mega.py")

from mega import Mega
import os
import shutil
from urllib.parse import urlparse, parse_qs
import urllib.parse

!rm -rf /content/unzips/
!rm -rf /content/zips/
!mkdir /content/unzips
!mkdir /content/zips

def sanitize_directory(directory):
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if os.path.isfile(file_path):
            if filename == ".DS_Store" or filename.startswith("._"):
                os.remove(file_path)
        elif os.path.isdir(file_path):
            sanitize_directory(file_path)

audio_zip = urlparse(url).path.split('/')[-2] + '.zip'
audio_zip_path = '/content/zips/' + audio_zip

if url != '':
    if "drive.google.com" in url:
        !gdown $url --fuzzy -O "$audio_zip_path"
    elif "mega.nz" in url:
        m = Mega()
        m.download_url(url, '/content/zips')
    else:
        !wget "$url" -O "$audio_zip_path"

    for filename in os.listdir("/content/zips"):
        if filename.endswith(".zip"):
            zip_file = os.path.join("/content/zips", filename)
            shutil.unpack_archive(zip_file, "/content/unzips", 'zip')

sanitize_directory("/content/unzips")

# Copy the unzipped audio files to the /content/drive/MyDrive/MDX_Colab/tracks folder
!mkdir -p /content/drive/MyDrive/MDX_Colab/tracks
for filename in os.listdir("/content/unzips"):
    if filename.endswith((".wav", ".mp3")):
        audio_file = os.path.join("/content/unzips", filename)
        destination_file = os.path.join("/content/drive/MyDrive/MDX_Colab/tracks", filename)
        shutil.copy2(audio_file, destination_file)
        if os.path.exists(destination_file):
            print(f"Copy successful: {destination_file}")
        else:
            print(f"Copy failed: {audio_file}")

!rm -r /content/unzips/
!rm -r /content/zips/


##Audio Isolation

In [ ]:
#@markdown #Загрузите файлы прямо в UVR
#@markdown Запустите эту ячейку, чтобы загрузить файлы с вокалом, которые вы хотите использовать (или ZIP-файлы, содержащие аудио), в свой Colab. <br>
#@markdown Альтернативно вы можете загрузить файлы с панели файлов Colab, но это должно быть более удобно. Этот метод может не работать на iOS.

from google.colab import files
from IPython.display import display, Javascript
import os
import shutil
import zipfile
import ipywidgets as widgets

# Create the target directory if it doesn't exist
target_dir = '/content/drive/MyDrive/MDX_Colab/tracks'
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

uploaded = files.upload()

for fn in uploaded.keys():
    # Check if the uploaded file is a zip file
    if fn.endswith('.zip'):
        # Write the uploaded zip file to the target directory
        zip_path = os.path.join(target_dir, fn)
        with open(zip_path, 'wb') as f:
            f.write(uploaded[fn])

        unzip_dir = os.path.join(target_dir, fn[:-4])  # Remove the .zip extension from the folder name

        # Extract the zip file
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(unzip_dir)

        # Delete the zip file
        if os.path.exists(zip_path):
            os.remove(zip_path)

        print('Архив "{name}" разархивирован и удален. Файлы находятся тут: {folder}'.format(name=fn, folder=unzip_dir))

        # Display copy path buttons for each extracted file
        for extracted_file in os.listdir(unzip_dir):
            extracted_file_path = os.path.join(unzip_dir, extracted_file)
            extracted_file_length = os.path.getsize(extracted_file_path)

            extracted_file_label = widgets.HTML(
                value='Разархивированный файл "{name}" весит {length} байт'.format(name=extracted_file, length=extracted_file_length)
            )
            display(extracted_file_label)

            extracted_file_path_text = widgets.HTML(
                value='Файл сохранен тут: <a href="{}" target="_blank">{}</a>'.format(extracted_file_path, extracted_file_path)
            )

            extracted_copy_button = widgets.Button(description='Copy')
            extracted_copy_button_file_path = extracted_file_path  # Make a local copy of the file path

            def copy_to_clipboard(b):
                js_code = '''
                    const el = document.createElement('textarea');
                    el.value = "{path}";
                    el.setAttribute('readonly', '');
                    el.style.position = 'absolute';
                    el.style.left = '-9999px';
                    document.body.appendChild(el);
                    el.select();
                    document.execCommand('copy');
                    document.body.removeChild(el);
                '''
                display(Javascript(js_code.format(path=extracted_copy_button_file_path)))

            extracted_copy_button.on_click(copy_to_clipboard)
            display(widgets.HBox([extracted_file_path_text, extracted_copy_button]))

        continue

    # For non-zip files
    # Save the file to the target directory
    file_path = os.path.join(target_dir, fn)
    with open(file_path, 'wb') as f:
        f.write(uploaded[fn])

    file_length = len(uploaded[fn])
    file_label = widgets.HTML(
        value='Пользователь загрузил файл "{name}" с весом {length} байт'.format(name=fn, length=file_length)
    )
    display(file_label)

    # Check if the uploaded file is a .pth or .index file
    if fn.endswith('.pth') or fn.endswith('.index'):
        warning_text = widgets.HTML(
            value='<b style="color: red;">Предупреждение:</b>Вы загружаете файл модели не в то место. Убедитесь, что он загружен в правильное место.'
        )
        display(warning_text)

    # Create a clickable path with copy button
    file_path_text = widgets.HTML(
        value='Файл сохранен тут: <a href="{}" target="_blank">{}</a>'.format(file_path, file_path)
    )

    copy_button = widgets.Button(description='Copy')
    copy_button_file_path = file_path  # Make a local copy of the file path

    def copy_to_clipboard(b):
        js_code = '''
            const el = document.createElement('textarea');
            el.value = "{path}";
            el.setAttribute('readonly', '');
            el.style.position = 'absolute';
            el.style.left = '-9999px';
            document.body.appendChild(el);
            el.select();
            document.execCommand('copy');
            document.body.removeChild(el);
        '''
        display(Javascript(js_code.format(path=copy_button_file_path)))

    copy_button.on_click(copy_to_clipboard)
    display(widgets.HBox([file_path_text, copy_button]))

# Remove the original uploaded files from /content/
for fn in uploaded.keys():
    if os.path.exists(os.path.join("/content/", fn)):
        os.remove(os.path.join("/content/", fn))

In [ ]:
#@markdown ### Print a list of tracks
for i in glob.glob('tracks/*'):
    print(os.path.basename(i))

In [ ]:
if not 'initialised' in globals():
    raise NameError('Пожалуйста, сначала запустите первую ячейку!! #scrollTo=H_cTbwhVq4K6')

#import all models metadata
import json
with open('model_data.json', 'r') as f:
  model_data = json.load(f)

# Modifiable variables
tracks_path = 'tracks/'
separated_path = 'separated/'

#@markdown ### Input track
#@markdown Enter any link/Filename (Upload your songs in tracks folder)
track = "Butterfly.wav" #@param {type:"string"}

#@markdown ---
#@markdown ### Models
ONNX = "MDX-UVR Ins Model Full Band 498 (HQ_2)" #@param ["off", "Karokee", "Karokee_AGGR", "Karokee 2", "baseline", "MDX-UVR Ins Model 415", "MDX-UVR Ins Model 418", "MDX-UVR Ins Model 464", "MDX-UVR Ins Model 496 - inst main-MDX 2.1", "Kim ft other instrumental model", "MDX-UVR Vocal Model 427", "MDX-UVR-Kim Vocal Model (old)", "MDX-UVR Ins Model Full Band 292", "MDX-UVR Ins Model Full Band 403", "MDX-UVR Ins Model Full Band 450 (HQ_1)", "MDX-UVR Ins Model Full Band 498 (HQ_2)"]
Demucs = 'off'#@param ["off","demucs_extra"]

#@markdown ---
#@markdown ### Parameters
denoise = False #@param {type:"boolean"}
normalise = True #@param {type:"boolean"}
#getting values from model_data.json related to ONNX var (model folder name)
amplitude_compensation = model_data[ONNX]["compensate"]
dim_f = model_data[ONNX]["mdx_dim_f_set"]
dim_t = model_data[ONNX]["mdx_dim_t_set"]
n_fft = model_data[ONNX]["mdx_n_fft_scale_set"]

mixing_algorithm = 'max_mag' #@param ["default","min_mag","max_mag"]
chunks = 55 #@param {type:"slider", min:1, max:55, step:1}
shifts = 10 #@param {type:"slider", min:0, max:10, step:0.1}

##validate values
track = track if 'http' in track else tracks_path+track
normalise = '--normalise' if normalise else ''
denoise = '--denoise' if denoise else ''

if ONNX == 'off':
    pass
else:
    ONNX = 'onnx/'+ONNX
if Demucs == 'off':
    pass
else:
    Demucs = 'model/'+Demucs+'.th'
#@markdown ---
#@markdown ### Stems
bass = False #@param {type:"boolean"}
drums = False #@param {type:"boolean"}
others = False #@param {type:"boolean"}
vocals = True #@param {type:"boolean"}
#@markdown ---
#@markdown ### Invert stems to mixture
invert_bass = False #@param {type:"boolean"}
invert_drums = False #@param {type:"boolean"}
invert_others = False #@param {type:"boolean"}
invert_vocals = True #@param {type:"boolean"}
invert_stems = []
stems = []
if bass:
    stems.append('b')
if drums:
    stems.append('d')
if others:
    stems.append('o')
if vocals:
    stems.append('v')

invert_stems = []
if invert_bass:
    invert_stems.append('b')
if invert_drums:
    invert_stems.append('d')
if invert_others:
    invert_stems.append('o')
if invert_vocals:
    invert_stems.append('v')

margin = 44100

###
# incompatibilities
###

console(f"python main.py --n_fft {n_fft} --dim_f {dim_f} --dim_t {dim_t} --margin {margin} -i \"{track}\" --mixing {mixing_algorithm} --onnx \"{ONNX}\" --model {Demucs} --shifts {round(shifts)} --stems {''.join(stems)} --invert {''.join(invert_stems)} --chunks {chunks} --compensate {amplitude_compensation} {normalise} {denoise}")

##**Объяснение настроек Colab для UVR**<br>

Обычно рекомендуются уже предоставленные значения по умолчанию. Однако, если вы хотите попробовать их настроить, вот объяснение:

*Алгоритм микширования* - max_mag - обычно для вокала (даёт больше всего остатков в инструментальных партиях), min_mag - для инструментальных партий (наиболее агрессивных), хотя "min_mag решает некоторые нежелательные вокальные звучания, но инструментальная [является] более приглушенной и менее детальной ". Также обратите внимание на «по умолчанию», поскольку оно находится между ними — то есть среднее (это также требуется для включенного Demucs, который работает только для вокальных моделей).<br>

*Чанки* — установите значение 55 или 40 (менее агрессивно), чтобы избежать случайного исчезновения некоторых инструментов.
Установите 1 для лучшей четкости. Это работает, по крайней мере, для инструментальной модели (дорожка 4:15, по крайней мере, для Tesla T4 (показано вверху), как правило, лучшего качества, но некоторые инструменты имеют тенденцию исчезать больше при использовании 1, чем 10. Для включенной Demucs и/или вокальной модели это может установите значение 10, если ваш трек длится менее 5:00 минут. Чем больше фрагментов, тем быстрее разделение до ~ 40. Для трека 4:15 максимально поддерживается 72, пока не появится ошибка распределения памяти (отключенные фрагменты также возвращают ошибку). <br>

*Смены* — можно установить максимум до 10, но это лишь немного увеличивает SDR, в то время как время обработки увеличивается в 1,7 раза для каждой смены и дает результат, аналогичный сдвигу 5.

*Нормализация* - "сначала нормализует весь входной сигнал, а затем возвращает пик волны к исходному. Это делает процесс разделения лучше, а также снижает уровень шума" (например, если вам нужны шумные хэты или большая компенсация амплитуды - отключите ее).
<br>

*Demucs* включен корректно работает с алгоритмом микширования, установленным по умолчанию, и только с вокальными моделями (Ким и 427). Это также единственный способ избавиться от шума моделей MDX. Включенная нормализация необходима (но эта конфигурация имеет немного больше голосовых остатков, чем инструментальная модель). Уменьшите количество фрагментов до 40, если у вас есть ONNXRuntimeError с включенным Demucs (для этого требуются меньшие фрагменты).
<br>

##**Рекомендуемые модели**<br>

Для вокала (по необработанному выходу SDR, без учета ручной очистки):
- Kim vocal 2 (меньше инструментальных остатков в голосовой основе)
- Kim vocal 1
<br>или альтернативные
- 427
- 406

Для лучшего ведущего вокала:
- Karaokee 2

Для лучшего бэк-вокала:
- [HP_KAROKEE-MSB2-3BAND-3090](https://colab.research.google.com/drive/16Q44VBJiIrXOgTINztVDVeb0XKhLKHwl?usp=sharing)

Довольно неудобно, что моделей VR-архитектуры здесь нет, и их приходится запускать через вышеупомянутый колаб, но на данный момент они не могут сосуществовать в одном колабе. В будущем я постараюсь найти лучшее решение.